In [3]:
import numpy as np
import re
import string
import pandas as pd
import nltk
from nltk.collocations import *
from pattern.en import lemma
import requests
import json
import time
import hashlib
import uuid
import csv
import phrasemachine
import time
from urllib.request import urlopen
from bs4 import BeautifulSoup
from urllib.request import Request
import difflib
from nltk.stem import WordNetLemmatizer 
from wechatpy.client import WeChatClient
import random
import http.client
import stardict
from stardict import LemmaDB
lemma = LemmaDB()
lemma.load('lemma.en.txt')
import urllib

In [4]:
GMAT_list = []

with open('GMAT_edited.txt','r',encoding='utf-8') as fhandle:
    for line in fhandle:
        GMAT_list.append(line.lower().rstrip())
    

word_pron_char_defin_cate = []   
df = pd.read_csv("stardict.csv",encoding = 'utf-8')

D:\apps\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Output v3

In [1]:
filename = input('enter the name of the v2file')

enter the name of the v2fileHeartistheonlyhuntervocabularycounts_v2


In [5]:
csvFile = open( filename + ".csv", "r",encoding = 'utf-8')
reader = csv.reader(csvFile)

# API id and key
app_id = '05d6c109'
app_key = '63bda9528e24dff26ca5dd695b008acf'

for i in reader:
    try:
        category = re.findall("[a-z]+\-?[a-z]+", df.loc[df['word']== i[0] ,'tag'].to_string())
        if i[0] in GMAT_list:
            category.append('gmat')
        cate = ",".join(category)
    
        try:
    
            word_id = i[0]
            strictMatch = 'false'

            url = 'https://od-api.oxforddictionaries.com:443/api/v2/entries/en-gb/' + word_id.lower() + '?strictMatch=' + strictMatch;

            r = requests.get(url, headers = {'app_id': app_id, 'app_key': app_key})
            pron = r.json()["results"][0]['lexicalEntries'][0]['entries'][0]['pronunciations'][0]['phoneticSpelling']
            char = r.json()["results"][0]['lexicalEntries'][0]['lexicalCategory']['text']
            defin = r.json()["results"][0]['lexicalEntries'][0]['entries'][0]['senses'][0]['definitions'][0]
            word_pron_char_defin_cate.append([i[0],'/'+pron+'/',char,defin,cate])
            time.sleep(10)
        except KeyError :
            pass
        continue
    except IndexError:
        pass
print('ready for chinese translation')

for i in word_pron_char_defin_cate:
    if i[2] == 'Noun':
        i[2] = 'n.'
    if i[2] == 'Adjective':
        i[2] = 'adj.'
    if i[2] == 'Verb':
        i[2] = 'v.'
    if i[2] == 'Adverb':
        i[2] = 'adv.'

# Baidu API for EN-CN Translate
appid = '20200627000507736'  # 填写你的appid
secretKey = 'G3ghonyD9dfI39NnFtGN'  # 填写你的密钥

httpClient = None
myurl = '/api/trans/vip/translate'

fromLang = 'en'   #原文语种
toLang = 'zh'   #译文语种

word_pron_char_defin_cate_cn = []
for i in word_pron_char_defin_cate:
    salt = random.randint(32768, 65536)
    sign = appid + i[0] + str(salt) + secretKey
    sign = hashlib.md5(sign.encode()).hexdigest()
    myurl = myurl + '?appid=' + appid + '&q=' + urllib.parse.quote(i[0]) + '&from=' + fromLang + '&to=' + toLang + '&salt=' + str(salt) + '&sign=' + sign

    try:
        httpClient = http.client.HTTPConnection('api.fanyi.baidu.com')
        httpClient.request('GET', myurl)

        # response是HTTPResponse对象
        response = httpClient.getresponse()
        result_all = response.read().decode("utf-8")
        result = json.loads(result_all)['trans_result'][0]['dst']
        i.append(result)
        word_pron_char_defin_cate_cn.append(i)


    except Exception as e:
        pass

fileObject = open(filename + '_v3.txt', 'w',encoding='utf-8')
for i in word_pron_char_defin_cate_cn:
    for j in i:
        fileObject.write(j)
        fileObject.write(' ')
    fileObject.write('\n')
fileObject.close()

ready for chinese translation
